# OpenStreetMap Data Case Study 

Submitted by : Tony Bastin 

## Map Area 

St. John's, NL, Canada

    *  https://www.openstreetmap.org/relation/2805426
    *  https://www.openstreetmap.org/export#map=10/47.4829/-52.8291 
    
St. John's is the town where I lived for more than 2 years while pursuing my Master's degree in Engineering. This project gives me a chance to check the data from OpenStreetMap.org and make improvements to it based on my knowledge of the place.


## Problems Encountered in the Map

After running the code in "Audit.ipynb" file, the following problems were identified in the osm file for St. John's :

    * Abrreviated and Incomplete street names
    * Inconsistent postal codes
    * Incorrect city names
    
### Abrreviated, Incorrect and Incomplete street names

Looking through the  results of  "Audit.ipynb" code, it was seen that some of the street name were abbreviated some were incorrect and some others were incomplete. The abrreviated one included 'Ave' for 'Avenue', 'Rd', 'Rd.'for 'Road' and 'st' for 'Street'. There was one incorrect street name where 'Extention' was used instead of  'Extension'. A few street names  were incomplete like simply stating 'Monkstown' for 'Monkstown Road'. The incomplete street names were a little hard to correct as I had to google these street names and findout the correct names. The following mapping dictionary and update_name() were used to correct the data before exporting to SQL.

    mapping = { "st" : "Street",
            "Rd" : "Road",
            "Rd.": "Road",
            "Ave": "Avenue",
            'Extention': "Extension",
            'Monkstown' : 'Monkstown Road',
            'Harvey': 'Harvey Road',
            'Hayward': 'Hayward Avenue', 
            'Larkhall': 'Larkhall Street',
            'Maxse': 'Maxse Street',
            'Monkstown': 'Monkstown Road',
            'Williams': 'Williams Heights',
            'catherine': 'catherine Street'
           
            }
            
     def update_name(name, mapping):

        m = street_type_re.search(name)
        if m:        
            st_type = m.group()
            if st_type in mapping:
                name = re.sub(street_type_re, mapping[st_type], name)

        return name   

### Inconsistent postal codes

The different postal codes in the osm file for St. John's were grouped according to thier lengths and is listed out using the "Audit.ipynb" code. It  was found that the postal codes had differnt length 3, 5, 6, 7 and 15. A few samples of the postal codes of different lengths are given below :

     3: {'A1N'},
     5: {'A1L 1'},
     6: {'A0A1W0', 'A1A0H5', .. }
     7: {'A0A 2M0', 'A0A 2R0', 'A0A 3K0', .. }
     15: {'A1B 1C2;A1B 2C3', 'A1B 2B6;A1B 2B7', 'A1B 2C6;A1B 4C6', 'A1B 3K8;A1B 3L3'}


The ideal postal code should have a length of 7. The lengths 3 and 5 indicate that the postal codes were not filled in completely. The length 15 indicate that 2 postal codes were listed instead of one. The above 2 defects were not corrected and left out as it is as the exact were values not clear. The postal codes with lenth 6 was modified into the ideal format by introducing a ' ' in the middle.

The following function was used to modify the postal codes.

    def update_post_code(post_code):
        
        if len(post_code) == 6:
            better_post_code = post_code[0:3]+" "+ post_code[3:]
            return  (better_post_code)
        else:
            return (post_code)


### Incorrect city names

St. John's is a small city and so in the map area a few more cities also appear. After auditing the city names it was found that "St.John's" was listed in 7 different ways and "Portugal Cove-St. Philip's" was listed in 6 different ways. The following mapping function was used to correct the city names.

    mapping = {           
                "Saint John's": "St. John's",
                "St john's": "St. John's",
                'St. John': "St. John's",
                "St. John's": "St. John's",
                'St. John´s': "St. John's",                   
                "st. John's": "St. John's",
                "st. john's": "St. John's",
                "Town of Portugal Cove - St. Philip's":"Portugal Cove-St. Philip's",
                "St. Phillips":"Portugal Cove-St. Philip's" ,
                "Portugal Cove - St. Philips": "Portugal Cove-St. Philip's",
                "Portugal Cove-St. Philip’s": "Portugal Cove-St. Philip's",
                'PORTUGAL COVE-ST PHILIPS': "Portugal Cove-St. Philip's",
                'St. Phillips': "Portugal Cove-St. Philip's"   

                }
                
 The city names were corrected using the code given below:
 
     def update_city_names(city_name,mapping):
        if city_name in mapping:        
            city_name = mapping[city_name]
        return city_name


## Overview of the Data
This section contains basic statistics about the dataset and the SQL queries used to gather them. 

### File sizes

    St. John's_NL_Canada.osm : 94,737 KB  
    St. John's.sqlite        :  164.3 KB  
    nodes.csv                : 38,231 KB  
    nodes_tags.csv           :    823 KB 
    ways.csv                 :  2,044 KB 
    ways_tags.csv            :  3,208 KB  
    way_nodes.csv            : 12,302 KB 

### Number of nodes

    SELECT COUNT(*) FROM nodes;  

    466957 

### Number of ways

    SELECT COUNT(*) FROM ways;

    34693   

### Number of unique users

    SELECT COUNT(DISTINCT(e.uid)) 
    FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;

    264 


### Top 5 contributing users

    SELECT e.user, COUNT(*) as num
    FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
    GROUP BY e.user
    ORDER BY num DESC
    LIMIT 5;      

    bgamberg     301092
    Adam Martin  73764
    Andrew_Finn  64966
    jfd553       4646
    Skybunny     4197
    
### Number of users appearing only once ﴾having 1 post﴿   

    SELECT COUNT(*)
    FROM (SELECT e.user, COUNT(*) as num
    FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
    GROUP BY e.user HAVING num=1) u;
    

## Additional ideas
   

### Top 10 appearing amenities in St.John's

    SELECT value, COUNT(*) as num
    FROM nodes_tags
    WHERE key = "amenity"
    GROUP BY value
    ORDER By num DESC
    LIMIT 5
    
    bench            237
    waste_basket     139
    restaurant        73 
    post_box          61 
    fast_food         59  
    place_of_worship  42
    fuel              29
    bicycle_parking   25
    cafe              24
    recycling         24

### Top 5 fast food chains in St. John's

    SELECT nodes_tags.value, COUNT(*) as num
    FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'fast_food')  i
    ON nodes_tags.id = i.id
    WHERE nodes_tags.key = 'name'
    GROUP BY nodes_tags.value
    ORDER By num DESC
    LIMIT 5;
    
    McDonald's    5
    Tim Horton's  5
    Mary Brown's  4
    A&W           3
    Dairy Queen   3
 
### Top 5 restaurants in St. John's 

    SELECT nodes_tags.value, COUNT(*) as num
    FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'fast_food')  i
    ON nodes_tags.id = i.id
    WHERE nodes_tags.key = 'name'
    GROUP BY nodes_tags.value
    ORDER By num DESC
    LIMIT 5;
    
    Boston Pizza      2
    Jungle Jim's      2
    Little Caesars    2
    Old Town Pizzeria 2 
    Subway            2
    
    
### Top cuisine in restaurants

    SELECT nodes_tags.value, COUNT(*) as num
    FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'restaurant')  i
    ON nodes_tags.id = i.id
    WHERE nodes_tags.key = 'cuisine'
    GROUP BY nodes_tags.value
    ORDER By num DESC
    LIMIT 1;
    
    
    chinese   6
    
    

### Top 5 gas stations in St. John's

    SELECT nodes_tags.value, COUNT(*) as num
    FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'fuel')  i
    ON nodes_tags.id = i.id
    WHERE nodes_tags.key = 'name'
    GROUP BY nodes_tags.value
    ORDER By num DESC
    LIMIT 5;


    Canadian Tire          2
    Esso                   2
    Ultramar               2
    Anderson Avenue Esso   1
    Blackmarsh Road Esso   1
    
### Banks with the most branches

    SELECT nodes_tags.value, COUNT(*) as num
    FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'bank')  i
    ON nodes_tags.id = i.id
    WHERE nodes_tags.key = 'name'
    GROUP BY nodes_tags.value
    ORDER By num DESC
    LIMIT 5;


    CIBC	           4
    Scotiabank	       4
    Bank of Montreal   2
    Royal Bank	       2
    Credit Union	   1
    
### Most followed religion in St. John's

    SELECT nodes_tags.value, COUNT(*) as num
    FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'place_of_worship')  i
    ON nodes_tags.id = i.id
    WHERE nodes_tags.key = 'religion'
    GROUP BY nodes_tags.value
    ORDER By num DESC
    LIMIT 1;
    
    christian   25
    
## Conclusion

The St. John's was wrangled for the purpose of this excersise. However, after queriying the dataset I understand that a lot of improvements can be made to this dataset in many places.

Script should be written to check and correct the OpenStreetMaps data automatically, this code should be monitored and improvements should be made to the scripts.

The problem for the implementation is that since the data is being entered by different users, for set of good scripts to evolve it will take a lot of time. 


## References

    * https://www.udacity.com/
    * https://www.w3schools.com/sql/
    * http://www.tutorialspoint.com/python/